In [1]:
import os
import configparser
import sys

import torch
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from scipy.optimize import minimize

# 設定の読み込み
config = configparser.ConfigParser()
config_path = "../config.ini"
config.read(config_path)
PROJECT_DIR = config["paths"]["project_dir"]
sys.path.append(PROJECT_DIR)

from src.bnn import BayesianMLPModel
from src.utils_experiment import generate_integer_samples, negate_function

In [2]:
import numpy as np
import torch

from src.utils_warcraft import navigate_through_matrix, manhattan_distance

# objective function for bortoch

- 入力：目的変数のサンプル
- 出力：評価値

In [28]:
class WarcraftObjectiveBoTorch:
    def __init__(
        self,
        weight_matrix: torch.tensor,
    ) -> None:
        self.weight_matrix = weight_matrix
        self.shape = weight_matrix.shape
        self.search_space_1d_dict = {
            -3: "oo", -2: "ab", -1: "ac", 0: "ad", 1: "bc", 2: "bd", 3: "cd"
        }
        self.reverse_search_space_1d_dict = {v: k for k, v in self.search_space_1d_dict.items()}

    def string_to_tensor(
        self,
        direction_matrix: np.ndarray
    ) -> torch.tensor:
        tensor_matrix = torch.zeros(self.shape, dtype=torch.int)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                tensor_matrix[i, j] = self.reverse_search_space_1d_dict.get(direction_matrix[i, j], 0.0)
        return torch.tensor(tensor_matrix, dtype=torch.float32)
    
    def tensor_to_string(
        self,
        x: torch.tensor
    ) -> np.ndarray:
        direction_matrix = np.zeros(self.shape, dtype=object)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                direction_matrix[i, j] = self.search_space_1d_dict.get(x[i, j].item())
        return direction_matrix

    def __call__(
        self,
        x: torch.tensor
    ) -> torch.Tensor:
        if type(x) == torch.Tensor:
            direction_matrix = self.tensor_to_string(x)
        else:
            direction_matrix = x

        start = (0, 0)
        goal = (self.shape[0]-1, self.shape[1]-1)

        history = navigate_through_matrix(direction_matrix, start, goal)

        if history:
            path_weight = sum(self.weight_matrix[point] for point in history)
            norm_const = manhattan_distance(start, goal)
            loss1 = 1 - (1 - manhattan_distance(history[-1], goal) / norm_const) + path_weight
        else:
            loss1 = 1 
        
        mask = direction_matrix != "oo"
        loss2 = self.weight_matrix[mask].sum()

        loss = loss1 + loss2
        score = -loss
        
        return score
    
    def visualize(
        self,
        x: torch.tensor
    ) -> None:
        direction_matrix = self.tensor_to_string(x)
        print(direction_matrix)
    

def generate_initial_data(
    objective_function: callable,
    dataset_size: int,
    shape: tuple[int, int],
) -> torch.tensor:
    values = torch.tensor([-3, -2, -1, 0, 1, 2, 3])
    n, m = shape
    X_train = values[torch.randint(0, len(values), (dataset_size, n, m))]
    y_train = torch.stack([objective_function(x) for x in X_train]).unsqueeze(-1)
    return X_train, y_train


weight_matrix = torch.Tensor([
    [0.1, 0.4, 0.8, 0.8],
    [0.2, 0.4, 0.4, 0.8],
    [0.8, 0.1, 0.1, 0.2],
])

In [4]:
weight_matrix = torch.Tensor([
    [0.1, 0.4, 0.8, 0.8],
    [0.2, 0.4, 0.4, 0.8],
    [0.8, 0.1, 0.1, 0.2],
])

objective_function = WarcraftObjectiveBoTorch(weight_matrix)
X_train, y_train = generate_initial_data(objective_function, 10, weight_matrix.shape)

print(X_train.shape, y_train.shape)

torch.Size([10, 3, 4]) torch.Size([10, 1])


# GP

In [33]:
import warnings
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

warnings.filterwarnings("ignore")

# Assuming your WarcraftObjectiveBoTorch class and other functions are defined as provided.

# Step 1: Define the Objective Function
weight_matrix = torch.Tensor([
    [0.1, 0.4, 0.8, 0.8],
    [0.2, 0.4, 0.4, 0.8],
    [0.8, 0.1, 0.1, 0.2],
])

objective_function = WarcraftObjectiveBoTorch(weight_matrix=weight_matrix)

# Step 2: Generate Initial Data
X_train, y_train = generate_initial_data(objective_function, 10, weight_matrix.shape)

# Flatten X_train from (10, 3, 4) to (10, 12) if shape is (3, 4)
n_samples = X_train.shape[0]
X_train_flat = X_train.view(n_samples, -1).float()  # Convert to torch.float32

# Step 3: Train the Gaussian Process Model
gp = SingleTaskGP(X_train_flat, y_train)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

# Repeat optimization for a specified number of iterations
n_iterations = 500  # Set the number of iterations

for iteration in range(n_iterations):
    print(f"Iteration {iteration + 1}/{n_iterations}")

    # Step 4: Define the Acquisition Function
    ucb = UpperConfidenceBound(gp, beta=0.1)

    # Step 5: Optimize the Acquisition Function
    candidate_flat, acq_value = optimize_acqf(
        acq_function=ucb,
        bounds=torch.tensor([[-3.0] * X_train_flat.shape[1], [3.0] * X_train_flat.shape[1]]),  # Search space bounds
        q=1,  # Number of candidates to propose
        num_restarts=5,  # Number of restarts during optimization
        raw_samples=20,  # Number of raw samples
    )

    # Round the candidate to ensure integer inputs
    candidate_flat = torch.round(candidate_flat)

    # Clip the candidate values to be within the valid range of search_space_1d_dict
    min_key, max_key = -3, 3
    candidate_flat = torch.clamp(candidate_flat, min=min_key, max=max_key)

    # Reshape the candidate back to the original shape
    candidate = candidate_flat.view(weight_matrix.shape)

    # Evaluate the new candidate
    y_new = objective_function(candidate)

    # Step 6: Update the Model
    X_train = torch.cat([X_train, candidate.unsqueeze(0)])
    y_train = torch.cat([y_train, y_new.unsqueeze(0).unsqueeze(-1)])  # Correct the dimension of y_new

    # Flatten X_train again for GP
    X_train_flat = X_train.view(X_train.shape[0], -1).float()  # Convert to torch.float32

    # Refit the GP model with updated data
    gp = SingleTaskGP(X_train_flat, y_train)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    # Print the result of the current iteration
    print(f"New candidate point: \n{candidate}, \nFunction value: {y_new.item()}\n\n")

print("Optimization completed.")
optim_idx = y_train.argmax()
print(f'Optimal solution: \n{X_train[optim_idx]}, \nFunction value: {y_train[optim_idx].item()}')

Iteration 1/500
New candidate point: 
tensor([[ 0.,  2.,  3., -1.],
        [-0.,  2., -2.,  0.],
        [-2.,  1., -2.,  2.]]), 
Function value: -6.100000381469727


Iteration 2/500
New candidate point: 
tensor([[ 1.,  0.,  2.,  3.],
        [-1., -3., -2., -2.],
        [-3., -2., -3.,  3.]]), 
Function value: -4.800000190734863


Iteration 3/500
New candidate point: 
tensor([[ 3., -1., -1.,  2.],
        [ 3., -1.,  3., -2.],
        [ 1., -3.,  2., -0.]]), 
Function value: -6.000000476837158


Iteration 4/500
New candidate point: 
tensor([[-0.,  2., -3., -1.],
        [ 2., -0.,  2.,  0.],
        [ 2., -0., -2., -2.]]), 
Function value: -5.300000190734863


Iteration 5/500
New candidate point: 
tensor([[-0., -2.,  2.,  1.],
        [ 0.,  1.,  2., -0.],
        [ 2.,  0.,  1.,  3.]]), 
Function value: -6.100000381469727


Iteration 6/500
New candidate point: 
tensor([[ 1.,  1.,  1., -0.],
        [-2.,  2., -0., -2.],
        [ 2.,  2., -2.,  2.]]), 
Function value: -6.1000003814

In [34]:
import plotly.graph_objs as go
import plotly.io as pio

# Assuming y_train is already defined and contains the history of function values

# Convert y_train to a numpy array for easy manipulation (if it's not already)
y_train_np = y_train.squeeze().numpy()  # Convert to 1D array if it's a 2D tensor

# Create an iteration index for plotting
iterations = list(range(1, len(y_train_np) + 1))

# Create the plot using Plotly
fig = go.Figure()

# Add the line plot for the objective values
fig.add_trace(go.Scatter(
    x=iterations,
    y=y_train_np,
    mode='markers',
    name='Objective Value',
    line=dict(color='royalblue', width=2),
    marker=dict(size=8)
))

# Update layout to add titles and labels
fig.update_layout(
    title='Optimization History',
    xaxis_title='Iteration',
    yaxis_title='Objective Value',
    template='plotly_white',
    font=dict(size=14),
)

# Show the plot
fig.show()

In [35]:
print("Optimization completed.")
optim_idx = y_train.argmax()
print(f'Optimal solution: \n{X_train[optim_idx]}, \nFunction value: {y_train[optim_idx].item()}')

Optimization completed.
Optimal solution: 
tensor([[-1., -3., -3., -3.],
        [-2.,  2., -1., -2.],
        [-1.,  2.,  2.,  0.]]), 
Function value: -4.09999942779541


# BayesianMLP (epoch=1,000)

In [9]:
import warnings

import torch
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound

from src.bnn import BayesianMLPModel
from src.bnn import fit_pytorch_model

warnings.filterwarnings("ignore")

# Assuming your WarcraftObjectiveBoTorch class and other functions are defined as provided.

# Step 1: Define the Objective Function
weight_matrix = torch.Tensor([
    [0.1, 0.4, 0.8, 0.8],
    [0.2, 0.4, 0.4, 0.8],
    [0.8, 0.1, 0.1, 0.2],
])

objective_function = WarcraftObjectiveBoTorch(weight_matrix=weight_matrix)

# Step 2: Generate Initial Data
X_train, y_train = generate_initial_data(objective_function, 10, weight_matrix.shape)

# Flatten X_train from (10, 3, 4) to (10, 12) if shape is (3, 4)
n_samples = X_train.shape[0]
X_train_flat = X_train.view(n_samples, -1).float()  # Convert to torch.float32

# Step 3: Train the Gaussian Process Model
model  = BayesianMLPModel(X_train_flat, y_train)
fit_pytorch_model(model, num_epochs=1000, learning_rate=0.01)

# Repeat optimization for a specified number of iterations
# n_iterations = 100  # Set the number of iterations
n_iterations = 1000  # Set the number of iterations

for iteration in range(n_iterations):
    print(f"Iteration {iteration + 1}/{n_iterations}")

    # Step 4: Define the Acquisition Function
    ucb = UpperConfidenceBound(model, beta=0.1)

    # Step 5: Optimize the Acquisition Function
    candidate_flat, acq_value = optimize_acqf(
        acq_function=ucb,
        bounds=torch.tensor([[-3.0] * X_train_flat.shape[1], [3.0] * X_train_flat.shape[1]]),  # Search space bounds
        q=1,  # Number of candidates to propose
        num_restarts=5,  # Number of restarts during optimization
        raw_samples=20,  # Number of raw samples
    )

    # Round the candidate to ensure integer inputs
    candidate_flat = torch.round(candidate_flat)

    print(candidate_flat.shape)

    # Clip the candidate values to be within the valid range of search_space_1d_dict
    min_key, max_key = -3, 3
    candidate_flat = torch.clamp(candidate_flat, min=min_key, max=max_key)

    # Reshape the candidate back to the original shape
    candidate = candidate_flat.view(weight_matrix.shape)

    # Evaluate the new candidate
    y_new = objective_function(candidate)

    # Step 6: Update the Model
    X_train = torch.cat([X_train, candidate.unsqueeze(0)])
    y_train = torch.cat([y_train, y_new.unsqueeze(0).unsqueeze(-1)])  # Correct the dimension of y_new

    # Flatten X_train again for GP
    X_train_flat = X_train.view(X_train.shape[0], -1).float()  # Convert to torch.float32

    # Refit the GP model with updated data
    model  = BayesianMLPModel(X_train_flat, y_train)
    fit_pytorch_model(model, num_epochs=1000, learning_rate=0.01)

    # Print the result of the current iteration
    print(f"New candidate point: \n{candidate}, \nFunction value: {y_new.item()}\n\n")

print("Optimization completed.")
optim_idx = y_train.argmax()
print(f'Optimal solution: \n{X_train[optim_idx]}, \nFunction value: {y_train[optim_idx].item()}')

Iteration 1/1000
torch.Size([1, 12])
New candidate point: 
tensor([[ 3., -1., -3., -1.],
        [ 1., -0., -2.,  2.],
        [-1.,  1.,  3., -1.]]), 
Function value: -5.300000190734863


Iteration 2/1000
torch.Size([1, 12])
New candidate point: 
tensor([[-3., -2.,  1., -0.],
        [-1.,  0., -1., -2.],
        [-2.,  2., -1.,  0.]]), 
Function value: -6.000000476837158


Iteration 3/1000
torch.Size([1, 12])
New candidate point: 
tensor([[-1., -1.,  2., -0.],
        [ 3., -1.,  0.,  1.],
        [-1.,  2., -1.,  0.]]), 
Function value: -6.100000381469727


Iteration 4/1000
torch.Size([1, 12])
New candidate point: 
tensor([[ 3.,  2.,  1., -1.],
        [-2., -0.,  0., -2.],
        [-2.,  3.,  0., -2.]]), 
Function value: -6.100000381469727


Iteration 5/1000
torch.Size([1, 12])
New candidate point: 
tensor([[ 1., -2., -1.,  0.],
        [ 3.,  1., -2., -1.],
        [-2.,  1., -3.,  2.]]), 
Function value: -6.000000476837158


Iteration 6/1000
torch.Size([1, 12])
New candidate poin

In [11]:
import plotly.graph_objs as go
import plotly.io as pio

# Assuming y_train is already defined and contains the history of function values

# Convert y_train to a numpy array for easy manipulation (if it's not already)
y_train_np = y_train.squeeze().numpy()  # Convert to 1D array if it's a 2D tensor

# Create an iteration index for plotting
iterations = list(range(1, len(y_train_np) + 1))

# Create the plot using Plotly
fig = go.Figure()

# Add the line plot for the objective values
fig.add_trace(go.Scatter(
    x=iterations,
    y=y_train_np,
    mode='markers',
    name='Objective Value',
    line=dict(color='royalblue', width=2),
    marker=dict(size=8)
))

# Update layout to add titles and labels
fig.update_layout(
    title='Optimization History',
    xaxis_title='Iteration',
    yaxis_title='Objective Value',
    template='plotly_white',
    font=dict(size=14),
)

# Show the plot
fig.show()

In [12]:
X_train[optim_idx]

tensor([[-2., -3., -3., -3.],
        [-1., -2., -2., -3.],
        [-3., -2., -1.,  2.]])

In [14]:
objective_function = WarcraftObjectiveBoTorch(weight_matrix)
objective_function.visualize(X_train[optim_idx])

[['ab' 'oo' 'oo' 'oo']
 ['ac' 'ab' 'ab' 'oo']
 ['oo' 'ab' 'ac' 'bd']]


In [15]:
weight_matrix

tensor([[0.1000, 0.4000, 0.8000, 0.8000],
        [0.2000, 0.4000, 0.4000, 0.8000],
        [0.8000, 0.1000, 0.1000, 0.2000]])

In [29]:
sol1 = np.array([
    ["ad", "oo", "oo", "oo"],
    ["bc", "ad", "oo", "oo"],
    ["oo", "bc", "ac", "ac"]
], dtype=object)

sol2 = np.array([
    ["ad", "oo", "oo", "oo"],
    ["bc", "ad", "oo", "oo"],
    ["oo", "bc", "ac", "ad"]
], dtype=object)

sol3 = np.array([
    ["bd", "oo", "oo", "oo"],
    ["bc", "ad", "oo", "oo"],
    ["oo", "bc", "ac", "ac"]
], dtype=object)

sol4 = np.array([
    ["bd", "oo", "oo", "oo"],
    ["bc", "ad", "oo", "oo"],
    ["oo", "bc", "ac", "ad"]
], dtype=object)


objective_function = WarcraftObjectiveBoTorch(weight_matrix)

for sol in [sol1, sol2, sol3, sol4]:
    print(objective_function(sol))

tensor(-2.1000)
tensor(-2.1000)
tensor(-2.2000)
tensor(-2.2000)


# BayesianMLP (epoch=10,000)

In [37]:
import warnings

import torch
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound

from src.bnn import BayesianMLPModel
from src.bnn import fit_pytorch_model

warnings.filterwarnings("ignore")

# Assuming your WarcraftObjectiveBoTorch class and other functions are defined as provided.

# Step 1: Define the Objective Function
weight_matrix = torch.Tensor([
    [0.1, 0.4, 0.8, 0.8],
    [0.2, 0.4, 0.4, 0.8],
    [0.8, 0.1, 0.1, 0.2],
])

objective_function = WarcraftObjectiveBoTorch(weight_matrix=weight_matrix)

# Step 2: Generate Initial Data
X_train, y_train = generate_initial_data(objective_function, 10, weight_matrix.shape)

# Flatten X_train from (10, 3, 4) to (10, 12) if shape is (3, 4)
n_samples = X_train.shape[0]
X_train_flat = X_train.view(n_samples, -1).float()  # Convert to torch.float32

# Step 3: Train the Gaussian Process Model
model  = BayesianMLPModel(X_train_flat, y_train)
fit_pytorch_model(model, num_epochs=10000, learning_rate=0.01)

# Repeat optimization for a specified number of iterations
# n_iterations = 100  # Set the number of iterations
n_iterations = 1000  # Set the number of iterations

for iteration in range(n_iterations):
    print(f"Iteration {iteration + 1}/{n_iterations}")

    # Step 4: Define the Acquisition Function
    ucb = UpperConfidenceBound(model, beta=0.1)

    # Step 5: Optimize the Acquisition Function
    candidate_flat, acq_value = optimize_acqf(
        acq_function=ucb,
        bounds=torch.tensor([[-3.0] * X_train_flat.shape[1], [3.0] * X_train_flat.shape[1]]),  # Search space bounds
        q=1,  # Number of candidates to propose
        num_restarts=5,  # Number of restarts during optimization
        raw_samples=20,  # Number of raw samples
    )

    # Round the candidate to ensure integer inputs
    candidate_flat = torch.round(candidate_flat)

    print(candidate_flat.shape)

    # Clip the candidate values to be within the valid range of search_space_1d_dict
    min_key, max_key = -3, 3
    candidate_flat = torch.clamp(candidate_flat, min=min_key, max=max_key)

    # Reshape the candidate back to the original shape
    candidate = candidate_flat.view(weight_matrix.shape)

    # Evaluate the new candidate
    y_new = objective_function(candidate)

    # Step 6: Update the Model
    X_train = torch.cat([X_train, candidate.unsqueeze(0)])
    y_train = torch.cat([y_train, y_new.unsqueeze(0).unsqueeze(-1)])  # Correct the dimension of y_new

    # Flatten X_train again for GP
    X_train_flat = X_train.view(X_train.shape[0], -1).float()  # Convert to torch.float32

    # Refit the GP model with updated data
    model  = BayesianMLPModel(X_train_flat, y_train)
    fit_pytorch_model(model, num_epochs=10000, learning_rate=0.01)

    # Print the result of the current iteration
    print(f"New candidate point: \n{candidate}, \nFunction value: {y_new.item()}\n\n")

print("Optimization completed.")
optim_idx = y_train.argmax()
print(f'Optimal solution: \n{X_train[optim_idx]}, \nFunction value: {y_train[optim_idx].item()}')

Iteration 1/1000
torch.Size([1, 12])
New candidate point: 
tensor([[ 1., -3.,  2.,  2.],
        [-1., -2.,  3., -1.],
        [ 3.,  3.,  2., -2.]]), 
Function value: -5.700000286102295


Iteration 2/1000
torch.Size([1, 12])
New candidate point: 
tensor([[-0., -2.,  0., -3.],
        [ 1.,  1., -1.,  3.],
        [-2., -1.,  0., -1.]]), 
Function value: -5.300000190734863


Iteration 3/1000
torch.Size([1, 12])
New candidate point: 
tensor([[-0., -0.,  2.,  1.],
        [ 3.,  0., -2., -0.],
        [ 3., -2., -1.,  0.]]), 
Function value: -6.100000381469727


Iteration 4/1000
torch.Size([1, 12])
New candidate point: 
tensor([[ 1.,  3.,  2.,  0.],
        [ 1., -3., -1.,  0.],
        [-1., -3.,  0., -1.]]), 
Function value: -5.600000381469727


Iteration 5/1000
torch.Size([1, 12])
New candidate point: 
tensor([[ 1., -1.,  3., -1.],
        [ 2.,  1.,  3., -0.],
        [-2.,  2., -1., -2.]]), 
Function value: -6.100000381469727


Iteration 6/1000
torch.Size([1, 12])
New candidate poin

In [38]:
import plotly.graph_objs as go
import plotly.io as pio

# Assuming y_train is already defined and contains the history of function values

# Convert y_train to a numpy array for easy manipulation (if it's not already)
y_train_np = y_train.squeeze().numpy()  # Convert to 1D array if it's a 2D tensor

# Create an iteration index for plotting
iterations = list(range(1, len(y_train_np) + 1))

# Create the plot using Plotly
fig = go.Figure()

# Add the line plot for the objective values
fig.add_trace(go.Scatter(
    x=iterations,
    y=y_train_np,
    mode='markers',
    name='Objective Value',
    line=dict(color='royalblue', width=2),
    marker=dict(size=8)
))

# Update layout to add titles and labels
fig.update_layout(
    title='Optimization History',
    xaxis_title='Iteration',
    yaxis_title='Objective Value',
    template='plotly_white',
    font=dict(size=14),
)

# Show the plot
fig.show()

In [39]:
X_train[optim_idx]

tensor([[ 3., -3., -3., -3.],
        [-3., -3., -3., -3.],
        [-3.,  3.,  3., -3.]])

In [40]:
objective_function.visualize(X_train[optim_idx])

[['cd' 'oo' 'oo' 'oo']
 ['oo' 'oo' 'oo' 'oo']
 ['oo' 'cd' 'cd' 'oo']]


In [41]:
optim_idx

tensor(80)